In [3]:
from email.policy import default
import json
import os
import pickle
import time
from contextlib import contextmanager
from typing import List, NoReturn, Optional, Tuple, Union
from rank_bm25 import BM25Okapi
import faiss
import numpy as np
import pandas as pd
from datasets import Dataset, concatenate_datasets, load_from_disk
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm.auto import tqdm
from transformers import AutoTokenizer
from elasticsearch import Elasticsearch
from elastic_setting import *


from retrieval_es import *

In [4]:
#  if __name__ == "__main__":

# import argparse

# parser = argparse.ArgumentParser(description="")
# parser.add_argument(
#     "--dataset_name", default="../data/train_dataset", type=str, help=""
# )
# parser.add_argument(
#     "--model_name_or_path",
#     default="klue/roberta-large",
#     type=str,
#     help="",
# )
# parser.add_argument("--data_path", default="../data", type=str, help="")
# parser.add_argument(
#     "--context_path", default="wikipedia_documents.json", type=str, help=""
# )
# parser.add_argument("--use_faiss", default=False, type=bool, help="")

# args = parser.parse_args()

# Test sparse
# org_dataset = load_from_disk(args.dataset_name)
dataset_name = "../data/train_dataset"
org_dataset = load_from_disk(dataset_name)
full_ds = concatenate_datasets(
    [
        org_dataset["train"].flatten_indices(),
        org_dataset["validation"].flatten_indices(),
    ]
)  # train dev 를 합친 4192 개 질문에 대해 모두 테스트
print("*" * 40, "query dataset", "*" * 40)
print(full_ds)
print(len(org_dataset["train"]),len(org_dataset["validation"]))


# tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, use_fast=False,)
retriever = ElasticRetrieval()

query = "대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?"


with timer("bulk query by exhaustive search"):
    df, retrieved_context = retriever.retrieve(full_ds, topk=10)
    df["correct"] = [original_context in context for original_context,context in zip(df["original_context"],df["context"])]
    print(
        "correct retrieval result by exhaustive search",
        f"{df['correct'].sum()}/{len(df)}",
        df["correct"].sum() / len(df),
    )

with timer("single query by exhaustive search"):
    scores, indices = retriever.retrieve(query)


Loading cached processed dataset at ../data/train_dataset/train/cache-9c5e152ac62642fb.arrow
Loading cached processed dataset at ../data/train_dataset/validation/cache-04cef27ebd362e38.arrow


**************************************** query dataset ****************************************
Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 4192
})
3952 240
Ping Elasticsearch : True
[query exhaustive search] done in 70.034 s


Sparse retrieval with Elasticsearch:  18%|█▊        | 736/4192 [00:00<00:00, 3678.60it/s]

tmp type: <class 'dict'>
tmp["context"] type: <class 'str'>
retrieved_context type: <class 'list'>
len retrieved_context: 10
len retrieved_context[0]: 862
tmp type: <class 'dict'>
tmp["context"] type: <class 'str'>
retrieved_context type: <class 'list'>
len retrieved_context: 10
len retrieved_context[0]: 621
tmp type: <class 'dict'>
tmp["context"] type: <class 'str'>
retrieved_context type: <class 'list'>
len retrieved_context: 10
len retrieved_context[0]: 845
tmp type: <class 'dict'>
tmp["context"] type: <class 'str'>
retrieved_context type: <class 'list'>
len retrieved_context: 10
len retrieved_context[0]: 609
tmp type: <class 'dict'>
tmp["context"] type: <class 'str'>
retrieved_context type: <class 'list'>
len retrieved_context: 10
len retrieved_context[0]: 609
tmp type: <class 'dict'>
tmp["context"] type: <class 'str'>
retrieved_context type: <class 'list'>
len retrieved_context: 10
len retrieved_context[0]: 677
tmp type: <class 'dict'>
tmp["context"] type: <class 'str'>
retrieved_

Sparse retrieval with Elasticsearch:  26%|██▋       | 1104/4192 [00:00<00:00, 3622.23it/s]

...

tmp type: <class 'dict'>
tmp["context"] type: <class 'str'>
retrieved_context type: <class 'list'>
len retrieved_context: 10
len retrieved_context[0]: 602
tmp type: <class 'dict'>
tmp["context"] type: <class 'str'>
retrieved_context type: <class 'list'>
len retrieved_context: 10
len retrieved_context[0]: 426
tmp type: <class 'dict'>
tmp["context"] type: <class 'str'>
retrieved_context type: <class 'list'>
len retrieved_context: 10
len retrieved_context[0]: 1076
tmp type: <class 'dict'>
tmp["context"] type: <class 'str'>
retrieved_context type: <class 'list'>
len retrieved_context: 10
len retrieved_context[0]: 1228
tmp type: <class 'dict'>
tmp["context"] type: <class 'str'>
retrieved_context type: <class 'list'>
len retrieved_context: 10
len retrieved_context[0]: 1176
tmp type: <class 'dict'>
tmp["context"] type: <class 'str'>
retrieved_context type: <class 'list'>
len retrieved_context: 10
len retrieved_context[0]: 655
tmp type: <class 'dict'>
tmp["context"] type: <class 'str'>
retriev

Sparse retrieval with Elasticsearch:  96%|█████████▌| 4005/4192 [00:01<00:00, 3577.80it/s]

tmp type: <class 'dict'>
tmp["context"] type: <class 'str'>
retrieved_context type: <class 'list'>
len retrieved_context: 10
len retrieved_context[0]: 575
tmp type: <class 'dict'>
tmp["context"] type: <class 'str'>
retrieved_context type: <class 'list'>
len retrieved_context: 10
len retrieved_context[0]: 922
tmp type: <class 'dict'>
tmp["context"] type: <class 'str'>
retrieved_context type: <class 'list'>
len retrieved_context: 10
len retrieved_context[0]: 1687
tmp type: <class 'dict'>
tmp["context"] type: <class 'str'>
retrieved_context type: <class 'list'>
len retrieved_context: 10
len retrieved_context[0]: 914
tmp type: <class 'dict'>
tmp["context"] type: <class 'str'>
retrieved_context type: <class 'list'>
len retrieved_context: 10
len retrieved_context[0]: 584
tmp type: <class 'dict'>
tmp["context"] type: <class 'str'>
retrieved_context type: <class 'list'>
len retrieved_context: 10
len retrieved_context[0]: 923
tmp type: <class 'dict'>
tmp["context"] type: <class 'str'>
retrieved

Sparse retrieval with Elasticsearch: 100%|██████████| 4192/4192 [00:01<00:00, 3590.36it/s]


tmp type: <class 'dict'>
tmp["context"] type: <class 'str'>
retrieved_context type: <class 'list'>
len retrieved_context: 10
len retrieved_context[0]: 485
tmp type: <class 'dict'>
tmp["context"] type: <class 'str'>
retrieved_context type: <class 'list'>
len retrieved_context: 10
len retrieved_context[0]: 703
tmp type: <class 'dict'>
tmp["context"] type: <class 'str'>
retrieved_context type: <class 'list'>
len retrieved_context: 10
len retrieved_context[0]: 1124
tmp type: <class 'dict'>
tmp["context"] type: <class 'str'>
retrieved_context type: <class 'list'>
len retrieved_context: 10
len retrieved_context[0]: 820
tmp type: <class 'dict'>
tmp["context"] type: <class 'str'>
retrieved_context type: <class 'list'>
len retrieved_context: 10
len retrieved_context[0]: 768
tmp type: <class 'dict'>
tmp["context"] type: <class 'str'>
retrieved_context type: <class 'list'>
len retrieved_context: 10
len retrieved_context[0]: 863
tmp type: <class 'dict'>
tmp["context"] type: <class 'str'>
retrieved

In [6]:
print(type(retrieved_context))
print(len(retrieved_context))

<class 'list'>
10


In [8]:
retrieved_context[0]

'본래 ‘호자주의’는 엔베르 호자 스스로가 자신의 사상에 대해 명명한 명칭이 아니며, 단지 알바니아를 외부자의 시점으로 관찰한 서구(주로 프랑스) 지식인 및 언론인들이 사용했던 용어이다. 엔베르 호자는 스스로의 사상을 단순히 마르크스-레닌주의의 원칙 고수일 뿐이라고 하였다. 이러한 원칙은 수정주의를 받아들인 중국공산당과 대립하기 전에 알바니아 내에서 윤곽이 잡혀져 있었다. 엔베르 호자가 스스로를 마오이스트라고 칭하지는 않은 것과 무관하게 호자주의는 1960년대 중화인민공화국의 문화대혁명에 영향을 받은 것이었으며, 이론적으로는 마오쩌둥 사상과 흡사했다. 1978년 이후 엔베르 호자는 마오쩌둥 사상을 강하게 비판하였고, 그 결과 호자 사상과 마오쩌둥 사상 사이의 차이점을 구분하는 것이 비교적 쉬워졌다. 그러나 그 차이점은 아주 세밀한 부분에서의 차이점이기 때문에 마르크스주의자들이 상당히 다루기 어려운 부분이라고 여겨지고 있다. 호자의 사상은 이러한 특징 외에도 기술주의(技術主義)에 대한 반대·극단적인 무신론·알바니아 민족주의·엄격한 금욕주의(禁慾主義)·지속적인 문화 혁명 강행이라는 요소도 존재하였다.\\n\\n호자주의는 이오시프 스탈린, 그리고 스탈린 지배 아래 있는 소련이라는 국가 개념을 엄격하게 고수하는 태도를 보이며, 1956년 이후 급격히 수정주의의 길로 접어든 소련과 그 영향권 아래에 있는 공산주의 단체를 모두 "수정주의적 마르크스주의"로 강렬히 비판한다.'

In [11]:
res_str = " ".join(retrieved_context)
# res_str

In [12]:
type(df)

pandas.core.frame.DataFrame

In [18]:
df.dtypes

question            object
id                  object
context_id          object
context             object
original_context    object
answers             object
correct               bool
dtype: object

In [20]:
df.head()

,question,id,context_id,context,original_context,answers,correct
0,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,14489,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,"{'answer_start': [235], 'text': ['하원']}",True
1,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,6795,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,"{'answer_start': [212], 'text': ['《경영의 실제》']}",True
2,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,52322,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,"{'answer_start': [510], 'text': ['백성']}",True
3,11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,22832,이 불상군은 1978년 발견되어 학계에 알려졌으며 봉황리 햇골산 중턱 두 곳에 약간...,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...","{'answer_start': [625], 'text': ['중국']}",True
4,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,2269,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,"{'answer_start': [30], 'text': ['4개']}",True


In [19]:
df["context"]

0       미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...
1       '근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...
2       강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...
3       이 불상군은 1978년 발견되어 학계에 알려졌으며 봉황리 햇골산 중턱 두 곳에 약간...
4       동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...
                              ...                        
4187    기원전 265년, 전단은 조나라의 군사를 거느리고 연나라를 공격하여 중양(中陽)을 ...
4188    현재 우리가 볼 수 있는 티칼의 모습은 펜실베이니아 대학교와 과테말라 정부의 협조 ...
4189    1940년 5월 1일 오전 창씨개명에 비협조적이라는 이유로 조선총독부 경무국에서 소...
4190    사이리 투팍의 아버지 망코 잉카 유팡키는 스페인 군대에 의해 황제의 자리에 오른 인...
4191    본래 ‘호자주의’는 엔베르 호자 스스로가 자신의 사상에 대해 명명한 명칭이 아니며,...
Name: context, Length: 4192, dtype: object